In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
import pickle
import random

In [3]:
emotions_labels = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

In [4]:
x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

# Para el perceptrón

In [5]:
# Cargar el modelo
model_perceptron = tf.keras.models.load_model('best_model_perceptron.keras')

# Realizar predicciones
y_pred_prob = model_perceptron.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Convertir etiquetas one-hot a formato de clase
y_test_classes = np.argmax(y_test, axis=1)

# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test_classes, y_pred)
print(f'Accuracy: {accuracy:.2f}')

219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step
Accuracy: 0.25


In [6]:
# Imprimir algunas predicciones
num_predictions = 5
for i in range(num_predictions):
    pred_label = emotions_labels[y_pred[i]]
    random_index = random.randint(0, len(y_pred)-1)
    actual_label = emotions_labels[y_test_classes[random_index]]
    print(f'Predicción {i+1}: {pred_label}')
    print(f'Clase real {i+1}: {actual_label}')
    print('---+---')

Predicción 1: happiness
Clase real 1: neutral
---+---
Predicción 2: happiness
Clase real 2: surprise
---+---
Predicción 3: happiness
Clase real 3: surprise
---+---
Predicción 4: happiness
Clase real 4: fear
---+---
Predicción 5: happiness
Clase real 5: happiness
---+---


# Para ELM.

In [7]:
class elm(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_neurons=1000, regressor=None):
        self.n_hidden_neurons = n_hidden_neurons
        self.regressor = regressor

    def _sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-x))
    
    def fit(self, X, y, binary_weights = False):
        # Dependiendo de la selección de pesos binarios, se eligen pesos aleatorios o no
        input_size = X.shape[1]

        if binary_weights:
            self.input_weights = np.random.choice([-1, 1], size=(input_size, self.n_hidden_neurons))
            self.biases = np.random.choice([-1, 1], size=self.n_hidden_neurons)
        else:
            self.input_weights = np.random.randn(input_size, self.n_hidden_neurons)
            self.biases = np.random.randn(self.n_hidden_neurons)
        
        # Salida de la capa oculta
        H = self._sigmoid(np.dot(X, self.input_weights) + self.biases)
        
        # Regresión usando regularización si se especifica
        self.regressor.fit(H, y)

        # Calcular las predicciones sobre el conjunto de entrenamiento
        y_pred_train = self.predict(X)
        y_true_labels = np.argmax(y, axis=1)
        y_pred_labels = np.argmax(y_pred_train, axis=1)
        
        accuracy_train = accuracy_score(y_true_labels, y_pred_labels)
        print(f"Accuracy en el conjunto de entrenamiento: {accuracy_train:.4f}")
        return self
    
    def predict(self, X):
        H = self._sigmoid(np.dot(X, self.input_weights) + self.biases)
        return self.regressor.predict(H)
    
    def save(self, filename):
        # pa guardar el modelo
        model_data = {
            'input_weights': self.input_weights,
            'biases': self.biases,
            'regressor': self.regressor
        }
        with open(filename, 'wb') as f:
            pickle.dump(model_data, f)
    
    def load(self, filename):
        # pa cargar el modelo
        with open(filename, 'rb') as f:
            model_data = pickle.load(f)
        self.input_weights = model_data['input_weights']
        self.biases = model_data['biases']
        self.regressor = model_data['regressor']

In [8]:
x_test_scaled = StandardScaler().fit_transform(x_test)

## Para EML con pesos no binarios.

### Sin regularización.

In [9]:
elm_no_reg = elm(n_hidden_neurons=1000, regressor=Ridge(alpha=0.0))
elm_no_reg.load('elm_no_reg.keras')

y_pred_prob = elm_no_reg.predict(x_test_scaled)
acc_loaded = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_prob, axis=1))
print(f"Accuracy del modelo cargado: {acc_loaded:.4f}")

Accuracy del modelo cargado: 0.2257


Sale aún más bajo que el perceptrón.
### Con regularización Ridge

In [10]:
elm_ridge = elm(n_hidden_neurons=1000, regressor=Ridge(alpha=0.1))
elm_ridge.load('elm_ridge.keras')

y_pred_prob = elm_ridge.predict(x_test_scaled)
acc_loaded = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_prob, axis=1))
print(f"Accuracy del modelo cargado: {acc_loaded:.4f}")

Accuracy del modelo cargado: 0.2213


Está saliendo inferior al perceptrón.
### Con regularización Lasso

In [11]:
elm_lasso = elm(n_hidden_neurons=1000, regressor=Lasso(alpha=0.1))
elm_lasso.load('elm_lasso.keras')

y_pred_prob = elm_lasso.predict(x_test_scaled)
acc_loaded = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_prob, axis=1))
print(f"Accuracy del modelo cargado: {acc_loaded:.4f}")

Accuracy del modelo cargado: 0.2503


Con Lasso parece tener el mismo desempeño que el perceptrón.
### Con regularización Elastic-net

In [12]:
elm_elastic = elm(n_hidden_neurons=1000, regressor=ElasticNet(alpha=0.1, l1_ratio=0.5))
elm_elastic.load('elm_elastic.keras')

y_pred_prob = elm_elastic.predict(x_test_scaled)
acc_loaded = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_prob, axis=1))
print(f"Accuracy del modelo cargado: {acc_loaded:.4f}")

Accuracy del modelo cargado: 0.2503


Como ya se estaba observando, las últimas dos regularizaciones tienen un efecto similar.
## Con pesos binarios.
### Sin regularización.

In [13]:
elm_no_reg_bin = elm(n_hidden_neurons=1000, regressor=Ridge(alpha=0.0))
elm_no_reg_bin.load('elm_no_reg_bin.keras')

y_pred_prob = elm_no_reg_bin.predict(x_test_scaled)
acc_loaded = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_prob, axis=1))
print(f"Accuracy del modelo cargado: {acc_loaded:.4f}")

Accuracy del modelo cargado: 0.2233


Parece tener un rendimento similar a su competencia de pesos continuos.
### Con regularización Ridge

In [14]:
elm_ridge_bin = elm(n_hidden_neurons=1000, regressor=Ridge(alpha=0.1))
elm_ridge_bin.load('elm_ridge_bin.keras')

y_pred_prob = elm_ridge_bin.predict(x_test_scaled)
acc_loaded = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_prob, axis=1))
print(f"Accuracy del modelo cargado: {acc_loaded:.4f}")

Accuracy del modelo cargado: 0.2233


No hay un efecto significativo al modelo sin regularización.
### Con regularización Lasso.

In [15]:
elm_lasso_bin = elm(n_hidden_neurons=1000, regressor=Lasso(alpha=0.1))
elm_lasso_bin.load('elm_lasso_bin.keras')

y_pred_prob = elm_lasso_bin.predict(x_test_scaled)
acc_loaded = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_prob, axis=1))
print(f"Accuracy del modelo cargado: {acc_loaded:.4f}")

Accuracy del modelo cargado: 0.2503


Definitivamente, se está comportando muy similar a su competencia de pesos continuos.
### Con regularización Elastic-net.

In [16]:
elm_elastic_bin = elm(n_hidden_neurons=1000, regressor=ElasticNet(alpha=0.1, l1_ratio=0.5))
elm_elastic_bin.load('elm_elastic_bin.keras')

y_pred_prob = elm_elastic_bin.predict(x_test_scaled)
acc_loaded = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_prob, axis=1))
print(f"Accuracy del modelo cargado: {acc_loaded:.4f}")

Accuracy del modelo cargado: 0.2503


El comportamiento con los datos de prueba parece no ser muy distinto entre los modelos de ELM, además el término de regularización no aporta demasiado, y en los que lo hace, con dificultades alcanza la precisión del MLP.
Aunque ciertamente el MLP requiere de un poco más de poder de cómputo debido al entrenamiento, la realidad es que es mínimo, y ya que está entrenado, sin dificultades alcanza la misma precisión que el modelo de ELM con la regularización más fuerte.

Una posible explicación de esto es que, al ser solo los indices correspondientes a los vectores de embeddings, al ser el ajuste correspondiente de la salida del encoder de la VQ-VAE, al estar en un espacio latente más reducido, se puede dar el caso en que dos rostros estén en una posición similar, eso llevaría a que su representación en el espacio latente fuera muy similar, haciendo corresponder el mismo codemap, y si estos dos rostros se etiquetaban en emociones distintas, entonces es donde empiezan los problemas.

En clase se nos mostró un ejemplo con imágenes de aves e incluso se nos mencionó que pasaba que con imágenes de distintas aves, el codemap quedaba muy similar, esto puede ser suficiente como para que, en un problema de clasificación, se logre una precisión tan baja.